__Accomplishments__

____________________________________
Accompli :
- Keep certifs
- Projets => Nombres de projets 
- +/- Verify skills
- Honors keep ( NLP +/- ou nombre )
- Keep languages
- Organizations => Nombre of organizations
____________________________________

In [36]:
import pandas as pd
df = pd.read_pickle("dummy.pkl")
df = df.drop(columns=["scraped_time"])
df = df.drop(columns=["interests"])
df = df.drop(columns=["search"])

In [37]:
json_array= []

for i in range(len(df)):
    json_array.append(df["accomplishments"][i])

store_list = []

for item in json_array:
    store_details = {"courses":None,"certifications":None, "projects":None,"languages":None,"nbre_languages":None,"honors":None,"organizations":None,"nbre_projet":None}
    store_details['certifications'] = len(item['certifications'])
    store_details['courses'] = len(item['courses'])
    store_details['projects'] = item['projects']
    store_details['languages'] = item['languages']
    store_details['honors'] = len(item['honors'])
    store_details['organizations'] = len(item['organizations'])
    store_details['nbre_projet'] = len(item['projects'])
    store_details['nbre_languages'] = len(item['languages'])
    store_list.append(store_details)

dtv=pd.DataFrame(store_list,index=None)
dtv=dtv.drop(columns="courses")
dtv=dtv.drop(columns="projects")
dtv=dtv.drop(columns="languages")
df=df.drop(columns="accomplishments")


In [38]:
df = pd.concat([df, dtv], axis=1, sort=False)

In [40]:
df["url"] = "*****************"
df["personal_info"] = "*****************"
df

,url,personal_info,skills,experiences,certifications,nbre_languages,honors,organizations,nbre_projet
0,*****************,*****************,"[{'name': 'AutoCAD', 'endorsements': '5'}, {'n...","{'jobs': [{'title': 'Architecte Principal', 'c...",0,0,0,0,0
1,*****************,*****************,"[{'name': 'Microsoft Office', 'endorsements': ...","{'jobs': [{'title': 'Founder and CEO', 'compan...",0,3,0,0,5
2,*****************,*****************,"[{'name': 'Microsoft Office', 'endorsements': ...","{'jobs': [{'title': 'Projet de fin d’études', ...",0,4,0,0,0
3,*****************,*****************,"[{'name': 'Java', 'endorsements': '6'}, {'name...","{'jobs': [{'title': 'Technical Lead', 'company...",0,1,0,0,4
4,*****************,*****************,"[{'name': 'R', 'endorsements': '7'}, {'name': ...",{'jobs': [{'title': 'PhD. Degree on Statistics...,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
9995,*****************,*****************,"[{'name': 'Python', 'endorsements': '37'}, {'n...",{'jobs': [{'title': 'Développeur Python/Devops...,0,2,0,0,0
9996,*****************,*****************,"[{'name': 'Management', 'endorsements': '1'}, ...",{'jobs': [{'title': 'Responsable Financement d...,0,0,0,0,0
9997,*****************,*****************,"[{'name': 'Java Enterprise Edition', 'endorsem...","{'jobs': [{'title': 'SAP Software Engineer', '...",0,0,0,0,0
9998,*****************,*****************,[],"{'jobs': [{'title': 'Médecin', 'company': 'Cna...",0,0,0,0,0
